In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()

In [ ]:
# define parameters for SCB's JSON API for population data
# don't forget to change the default 'px' response format to 'json' bottom of the pop_query data struct 

pop_url = 'http://api.scb.se/OV0104/v1/doris/en/ssd/START/BE/BE0101/BE0101A/FolkmangdNov'

pop_query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
# define parameters for SCB's JSON API for death data
# don't forget to change the default 'px' response format to 'json' bottom of the dead_query data struct

dead_url = 'http://api.scb.se/OV0104/v1/doris/en/ssd/START/BE/BE0101/BE0101I/DodaFodelsearK'
dead_query = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "2002",
          "2003",
          "2004",
          "2005",
          "2006",
          "2007",
          "2008",
          "2009",
          "2010",
          "2011",
          "2012",
          "2013",
          "2014",
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
### function to fetch SCB population or death data, returns 3 DataFrames w. different age binnings ### 

def fetch_scb_data(url,query,param='pop'):
    
    # call the API, check HTML status code (should be 200 for success)

    r= requests.post(url,json=query)
    print ('HTML status code fetching {}: '.format(param),r.status_code)
    
    # extract the json data section from the response

    json_data = r.json()['data'] # get the data section (skip metadata)
    json_data[0] # check the first record to figure out the structure of the data

    # we get two dictionaries : 'key' and 'values'
    # 'key' attributes are : 'geo' (all of sweden), 'age','gender','year'
    # 'values' attribute is : 'population' / 'dead'

    print ('record layout for {}: '.format(param),json_data[0])
    
    # create a Pandas DataFrame from the list of dictionaries, and massage it to a decent structure

    age_df = pd.DataFrame.from_dict(json_data)
    age_df[['area','age','gender','year']] = age_df['key'].to_list() # split keys to separate columns

    age_df[param] = age_df['values'].apply(
        lambda x : x[0]).astype(int) # extract size from the values list

    age_df['num_age'] = age_df['age'].apply(
        lambda x :x.replace('+','')).astype(int) # create numeric age column

    age_df = age_df.drop(['key','values','area'],axis=1) # delete unwanted columns

    # combine the two genders
    age_df = age_df.groupby(['year','num_age']).sum()

    # bin the data to the same age bins that are used in SCB's prel. death data
    age_df['scb_prel_age_bin'] = pd.cut(age_df.index.get_level_values(1),[-1,64,79,89,200],
                                        labels=['-64','65-79','80-89','90+'])

    # and finally, for each year, combine the 1 year age bin sums to match the 4 age bins of the prel death data

    age_scb_prel_df = age_df.groupby([age_df.index.get_level_values(0),
                                                      'scb_prel_age_bin']).sum()
    
    # create a df with total yearly values
    df = age_scb_prel_df.groupby('year').sum()
    
    
    

    return age_df,age_scb_prel_df,df
    

In [ ]:
### get the population & death data from SCB ###
pop_age_df,pop_age_scb_prel_df,pop_df = fetch_scb_data(pop_url,pop_query,param='pop')
dead_age_df,dead_age_scb_prel_df,dead_df = fetch_scb_data(dead_url,dead_query,param='dead')


In [ ]:
### combine the pop & death data to 3 separate, complete df's with: ###
### 1) 1 year age bins; 2) 4 age bins as with scb prel death data, 3) no age bins ###

one_year_binned = pd.DataFrame(pop_age_df['pop']).join(dead_age_df['dead'])
scb_prel_binned = pd.DataFrame(pop_age_scb_prel_df['pop']).join(dead_age_scb_prel_df['dead'])
no_bins = pd.DataFrame(pop_df['pop']).join(dead_df['dead'])


In [ ]:
### now we need to fetch the 2020 prel death data which do no yet reside in the scb database

scb_prel_url = 'https://www.scb.se/hitta-statistik/statistik-efter-amne/befolkning/'\
'befolkningens-sammansattning/befolkningsstatistik/pong/tabell-och-diagram/preliminar-statistik-over-doda/'

scb_prel_file = pd.ExcelFile(scb_prel_url)

In [ ]:
# check the sheet names #
# Tabell 1 contains the daily, non age-binned data #
# Tabell 7 contains the week number based, age-binned data #
# Note that the last week, 53, reaches to 2021-01-03, thus the age based total is a bit higher than the daily sum #

scb_prel_file.sheet_names

In [ ]:
# parse the prel daily death data #

scb_prel_daily = scb_prel_file.parse(sheet_name='Tabell 1',skiprows=6,usecols=range(10))

scb_prel_daily


In [ ]:
# get the deaths occurred on unknown days #
unknown_death_day = scb_prel_daily.iloc[-1,1:]

# remove the unknown deaths from the df and drop the DagMånad column #

scb_prel_daily = scb_prel_daily.iloc[:-1,1:]
scb_prel_daily

In [ ]:
# let's allocate the unknown deaths proportionally to the days of year #

daily_proportion_of_total_deaths = scb_prel_daily / scb_prel_daily.sum()
deaths_to_add = daily_proportion_of_total_deaths * unknown_death_day
scb_prel_daily += deaths_to_add
scb_prel_daily

In [ ]:
total_deaths = scb_prel_daily.sum()
total_deaths

In [ ]:
### let's add the prel 2020 daily death total to no_bins df ###
### notice that the 2020 total death toll calculated by daily deaths is not identical to ###
### the total calculated by weekly sums below due to week 53 stretching to Jan 3'd ###
### the difference is ~800 more deaths for the weekly calculation ###

no_bins.loc['2020','dead'] = total_deaths['2020']
no_bins

In [ ]:
### next, let's get the weekly, age-binned deaths ###

scb_prel_age = scb_prel_file.parse('Tabell 7',skiprows=8,usecols=range(11,19))
scb_prel_age.tail()

In [ ]:
# unknown death date for the age-binned prel. data #
unknown_age_day = scb_prel_age.loc[53]

# drop the unknowns from the df

scb_prel_age = scb_prel_age.loc[:52]
scb_prel_age.tail()

In [ ]:
### add the unknowns proportionally to each week ###
### the sum total will be about 800 higher than that for the daily sum, ###
### since the week 53 stretches to 2021-01-03 ###

weekly_prop = scb_prel_age / scb_prel_age.sum()
weekly_add_ons = weekly_prop * unknown_age_day

scb_prel_age += weekly_add_ons
scb_prel_age.sum(axis=1).sum()

In [ ]:
# set index to reflect week numbers #
scb_prel_age.index = range(1,54)
scb_prel_age.index.name = 'week_nr'
scb_prel_age.head()

In [ ]:
### function to combine gender specific deaths per age group to age grp total ###

def combine_ages(col):
    
    class AgeError(Exception):
        pass
    
    if '0-64' in col:
        return '-64'
    if '65-79' in col:
        return '65-79'
    if '80-89' in col:
        return '80-89'
    if '90+' in col:
        return '90+'
    else:
        return AgeError('no such age group')
    

In [ ]:
scb_prel_age = scb_prel_age.groupby(combine_ages,axis=1).sum()
scb_prel_age.tail()

In [ ]:
scb_age_2020_total = scb_prel_age.sum()
scb_age_2020_total

In [ ]:
### this is to allow for removal of the about 800 extra deaths that occur during 2021 ###
### but are included by the week-by-week data in the age grp prel.death calculations ###

REMOVE_2021 = True
deaths_occuring_2021 = 800

if REMOVE_2021:
    age_grp_2020_prop = scb_age_2020_total / scb_age_2020_total.sum()
    removals = deaths_occuring_2021 * age_grp_2020_prop
    scb_age_2020_total -= removals
    
scb_age_2020_total

In [ ]:
### now, let's add the 2020 prel death data to the age binned death dataframe from above ###

# first we need a multi-index for 2020 for the 4 age groups # 

idx = pd.MultiIndex.from_arrays([ ['2020'] * len (scb_prel_age.columns), scb_prel_age.columns ])

scb_age_2020_total.index = idx
scb_age_2020_total.name='dead'
scb_age_2020_total = pd.DataFrame(scb_age_2020_total)
scb_age_2020_total

In [ ]:
scb_prel_binned

In [ ]:
### and now let's add 2020 to the scb_prel_binned_df ###

scb_prel_binned.at['2020','dead'] = scb_age_2020_total
scb_prel_binned

In [ ]:
### note that the 2020 total, when computed week based, is about 800 over the daily total ###
### due to week 53 extending to Jan 3'd ###
### The boolean REMOVE_2021 is set to remove the 2021 deaths from the age grp based total ###

scb_prel_binned.groupby('year').sum()

In [ ]:
### now, let's add age group base mortality ###
scb_prel_binned['mortality'] = scb_prel_binned['dead'] / scb_prel_binned['pop']
scb_prel_binned

In [ ]:
### let's define 2019 as our standard population, for age adjusted overall mortality ###
### and then add 'standardized deaths', that is, absolute deaths based on ACTUAL yearly mortality ###
### acting on the standard population. I.e we are computing the number absolute deaths given actual ###
### mortality but as if the population age group sizes had been equal to that of the std year (2019) ###

std_pop = scb_prel_binned.loc['2019','pop']

scb_prel_binned['std_deaths'] = scb_prel_binned['mortality'] * std_pop
scb_prel_binned

In [ ]:
### now, to compute age adj mortality, for each year we sum the age_group std_deaths ###
### and divide by total std_population ###

age_adj_mortality = scb_prel_binned.groupby('year').sum()['std_deaths'] / std_pop.sum()
age_adj_mortality

In [ ]:
### lets compute non-age-adj mortality
no_bins['mortality'] = no_bins['dead'] / no_bins['pop']
no_bins

In [ ]:
### age adj Mortality baselines ###

age_adj_grp_mort_base_15_18 = scb_prel_binned.loc['2015':'2018','mortality'].groupby('scb_prel_age_bin').mean()
age_adj_grp_mort_base_15_19 = scb_prel_binned.loc['2015' :'2019','mortality'].groupby('scb_prel_age_bin').mean()

print ('base_15_18_mortality :',age_adj_grp_mort_base_15_18)
print ()
print ('base_15_19_mortality :',age_adj_grp_mort_base_15_19)

In [ ]:
### compute expected age grp deaths given mortality per age group is baseline mortality ###
scb_prel_binned['exp_dead_base_15_18'] = scb_prel_binned['pop'] * age_adj_grp_mort_base_15_18
scb_prel_binned['exp_dead_base_15_19'] = scb_prel_binned['pop'] * age_adj_grp_mort_base_15_19

In [ ]:
### compute excess age grp deaths as diff actual age grp deaths - expected age grp deaths ### 
scb_prel_binned['excess_15_18'] = scb_prel_binned['dead'] - scb_prel_binned['exp_dead_base_15_18']
scb_prel_binned['excess_15_19'] = scb_prel_binned['dead'] - scb_prel_binned['exp_dead_base_15_19'] 

scb_prel_binned

In [ ]:
### compute yearly sums of total excess deaths based on age groups ###
age_grp_based_excess_deaths = scb_prel_binned.groupby('year')[['excess_15_18','excess_15_19']].sum()
age_grp_based_excess_deaths

In [ ]:
### compute non-age-adj baselines ###
mort_15_18_base = no_bins.loc['2015':'2018']['mortality'].mean()
mort_15_19_base = no_bins.loc['2015' : '2019']['mortality'].mean()

print ('non-age-adj-mort_base_15_18 :',mort_15_18_base)
print ()
print ('non-age_adj-mort_base_15_19 :',mort_15_19_base)

In [ ]:
### compute expected deaths given these baselines ###
no_bins['exp_dead_base_15_18'] = no_bins['pop'] * mort_15_18_base
no_bins['exp_dead_base_15_19'] = no_bins['pop'] * mort_15_19_base

no_bins

In [ ]:
### compute excess deaths given the two expectations ###

no_bins['excess_15_18'] = no_bins['dead'] - no_bins['exp_dead_base_15_18']
no_bins['excess_15_19'] = no_bins['dead'] - no_bins['exp_dead_base_15_19']

no_bins

In [ ]:
### finally, let's compute the excess for absolute deaths, with the same baseline years ###
abs_base_15_18 = no_bins.loc['2015' : '2018','dead'].mean()
abs_base_15_19 = no_bins.loc['2015' : '2019','dead'].mean()

print ('abs_base_15_18 :',abs_base_15_18)
print ('abs_base_15_19 :',abs_base_15_19)

In [ ]:
abs_excess = pd.DataFrame({'15_18_base' : no_bins['dead'] - abs_base_15_18,
                          '15_19_base' : no_bins['dead'] - abs_base_15_19})
abs_excess


In [ ]:
### summary of excess deaths, computed in different ways, with two different baselines (15-18,15-19) ###

print ('\n2020 excess deaths :\n'.upper())

print ('Absolute excess deaths cmp baseline 15-18 :',abs_excess.loc['2020','15_18_base'].astype(int))
print ('\nAbsolute excess deaths cmp baseline 15-19 :',abs_excess.loc['2020','15_19_base'].astype(int))
print ('\nPopulation adjusted excess deaths cmp baseline 15-18 :',no_bins.loc['2020','excess_15_18'].astype(int))
print ('\nPopulation adjusted excess deaths cmp baseline 15-19 :',no_bins.loc['2020','excess_15_19'].astype(int))

print ('\nAge adjusted excess deaths cmp baseline 15-18 :',
       age_grp_based_excess_deaths.loc['2020','excess_15_18'].astype(int))

print ('\nAge adjusted excess deaths cmp baseline 15-19 :',
       age_grp_based_excess_deaths.loc['2020','excess_15_19'].astype(int))


